In [1]:
# Base & Visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.model_selection import cross_val_score

# Oversampling
from imblearn.combine import  SMOTETomek

# Modeling
from catboost import CatBoostClassifier, CatBoostRegressor
from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.ensemble import ExtraTreesClassifier, ExtraTreesRegressor
from sklearn.linear_model import LogisticRegression

# Scoring
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report

# BayesianOptimizer
from bayes_opt import BayesianOptimization

# Saving Model
import joblib

# Data Generate

In [2]:
# Data Load
train = pd.read_csv('data/part/one-hot_encoding/iterative/part_one_iter_train.csv')
label = pd.read_csv('data/raw_label.csv')
test = pd.read_csv('data/one_hot_test.csv')

train_x = np.array(train)
train_y = np.array(label)
test_x = np.array(test)

In [3]:
# Data split
x_train, x_val, y_train, y_val = train_test_split(train_x, train_y, test_size = 0.3, random_state = 17, stratify = train_y)

In [4]:
# Oversampling - train
tomek = SMOTETomek(random_state = 17, n_jobs = -1)
tomek_x_train, tomek_y_train = tomek.fit_sample(x_train, y_train)
print(tomek_x_train.shape, tomek_y_train.shape)

(17868, 53) (17868,)


In [5]:
# Making tomek_s_train & x_s_val for Task2
tomek_s_train = np.array(pd.DataFrame(tomek_x_train, columns = train.columns).loc[:,test.columns])
x_s_val = np.array(pd.DataFrame(x_val, columns = train.columns).loc[:,test.columns])

# Generate Hyperparameters by Model

In [6]:
# Catboost
cb_pbounds = { 'n_estimators' : (50, 500),
               'learning_rate' : (0.001, 0.5),
               'max_depth' : (3, 10),
               'random_strength' : (0, 0.9),
               'l2_leaf_reg' : (1, 50) }

# XGB
xgb_pbounds = { 'n_estimators' : (50, 500),
                'learning_rate' : (0.001, 0.5),
                'max_depth' : (3, 10),
                'subsample' : (0.7, 1),
                'colsample_bytree' : (0.7, 1),
                'reg_alpha' : (0.01, 10),
                'reg_lambda' : (0.01, 10),
                'gamma' : (0.01, 10),
                'min_child_weight' : (1, 15) }

# LGBM
lgbm_pbounds = { 'n_estimators' : (50, 500),
                 'learning_rate' : (0.001, 0.5),
                 'max_depth' : (3, 10),
                 'min_child_samples' : (5,50),
                 'subsample' : (0.7, 1),
                 'colsample_bytree' : (0.7, 1),
                 'reg_alpha' : (0.01, 10),
                 'reg_lambda' : (0.01, 10),
                 'min_child_weight' : (1, 15),
                 'min_split_gain' : (10,100) }

# ExtraTree
et_pbounds = { 'n_estimators' : (50, 500), 'max_depth' : (3, 10),
               'min_samples_split' : (1, 15), 'min_samples_leaf' : (2, 15) }

# Logistic
lr_pbounds = { 'max_iter' : (50, 500), 'C' : (0.01, 1) }

# Task1 : Making a Label predict_proba at y_train

### Generate Functoin for Bayesian Optimizer (Classifier)

In [10]:
def cb_clf_opt(n_estimators, learning_rate, max_depth, random_strength, l2_leaf_reg):
    
    params = { 'n_estimators' : int(round(n_estimators)),
               'learning_rate' : learning_rate,
               'max_depth' : int(round(max_depth)),
               'random_strength' : random_strength,
               'l2_leaf_reg' : max_iter}
    
    cb = CatBoostClassifier(**params)
    skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 17)
    score = cross_val_score(cb, tomek_x_train, tomek_y_train, scoring = 'f1_macro', cv = skf, n_jobs = -1)
    
    return np.mean(score)

def xgb_clf_opt(n_estimators, learning_rate, max_depth, subsample, colsample_bytree, reg_alpha, reg_lambda, gamma, min_child_weight):
    
    params = { 'n_estimators' : int(round(n_estimators)),
               'learning_rate' : learning_rate,
               'max_depth' : int(round(max_depth)),
               'subsample' : subsample,
               'colsample_bytree' : colsample_bytree,
               'reg_alpha' : reg_alpha,
               'reg_lambda' : reg_lambda,
               'gamma' : gamma,
               'min_child_weight' : min_child_weight, 'n_jobs' : -1 }
    
    xgb = XGBClassifier(**params)
    skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 17)
    score = cross_val_score(xgb, tomek_x_train, tomek_y_train, scoring = 'f1_macro', cv = skf, n_jobs = -1)
    
    return np.mean(score)

def lgbm_clf_opt(n_estimators, learning_rate, max_depth, min_child_samples, subsample, colsample_bytree, 
                 reg_alpha, reg_lambda, min_child_weight, min_split_gain):
    
    params = { 'n_estimators' : int(round(n_estimators)),
               'learning_rate' : learning_rate,
               'max_depth' : int(round(max_depth)),
               'min_child_samples' : int(round(min_child_samples)),
               'subsample' : subsample,
               'colsample_bytree' : colsample_bytree,
               'reg_alpha' : reg_alpha,
               'reg_lambda' : reg_lambda,
               'min_child_weight' : min_child_weight,
               'min_split_gain' : min_split_gain, 'n_jobs' : -1 }
    
    lgbm = LGBMClassifier(**params)
    skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 17)
    score = cross_val_score(lgbm, tomek_x_train, tomek_y_train, scoring = 'f1_macro', cv = skf, n_jobs = -1)
    
    return np.mean(score)

def et_clf_opt(n_estimators, max_depth, min_samples_split, min_samples_leaf):
    
    params = { 'n_estimators' : int(round(n_estimators)),
               'max_depth' : int(round(max_depth)),
               'min_samples_split' : int(round(min_samples_split)),
               'min_samples_leaf' : int(round(min_samples_leaf)), 'n_jobs' : -1 }
    
    et = ExtraTreesClassifier(**params)
    skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 17)
    score = cross_val_score(et, tomek_x_train, tomek_y_train, scoring = 'f1_macro', cv = skf, n_jobs = -1)
    
    return np.mean(score)

def lr_clf_opt(n_estimators, C):
    
    params = { 'max_iter' : int(round(max_iter)), 'C' : C, 'n_jobs' : -1 }
    
    lr = LogisticRegression(**params)
    skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 17)
    score = cross_val_score(lr, tomek_x_train, tomek_y_train, scoring = 'f1_macro', cv = skf, n_jobs = -1)
    
    return np.mean(score)

### Generate BayesianOptimization

In [11]:
BO_cb_clf = BayesianOptimization(f = cb_clf_opt, pbounds = cb_pbounds, random_state=17)
BO_xgb_clf = BayesianOptimization(f = xgb_clf_opt, pbounds = xgb_pbounds, random_state=17)
BO_lgbm_clf = BayesianOptimization(f = lgbm_clf_opt, pbounds = lgbm_pbounds, random_state=17)
BO_et_clf = BayesianOptimization(f = et_clf_opt, pbounds = et_pbounds, random_state=17)
BO_lr_clf = BayesianOptimization(f = lr_clf_opt, pbounds = lr_pbounds, random_state=17)

### Optimizer Maximize

In [13]:
# Catboost Maximize
BO_cb_clf.maximize(init_points=50, n_iter=150, acq = 'poi')

|   iter    |  target   | l2_lea... | learni... | max_depth | n_esti... | random... |
-------------------------------------------------------------------------------------
| 2         | 0.9739    | 47.34     | 0.03096   | 9.048     | 444.8     | 0.04607   |
| 3         | 0.9752    | 32.97     | 0.2763    | 7.183     | 267.6     | 0.2547    |
| 4         | 0.974     | 15.59     | 0.2812    | 5.772     | 404.9     | 0.3766    |
| 5         | 0.9728    | 8.051     | 0.0763    | 3.387     | 373.1     | 0.2631    |
| 6         | 0.9731    | 10.74     | 0.4159    | 6.976     | 87.05     | 0.4905    |
| 7         | 0.9728    | 8.789     | 0.3387    | 3.829     | 250.2     | 0.7992    |
| 8         | 0.9734    | 40.07     | 0.03492   | 9.726     | 346.6     | 0.6469    |
| 9         | 0.9728    | 37.44     | 0.444     | 3.938     | 399.6     | 0.7542    |
| 10        | 0.9581    | 41.64     | 0.01555   | 5.733     | 246.8     | 0.4686    |
| 11        | 0.968     | 25.15     | 0.06608   | 9.89

| 59        | 0.9693    | 35.76     | 0.03196   | 9.832     | 185.8     | 0.5912    |
| 60        | 0.975     | 39.98     | 0.4377    | 7.942     | 348.5     | 0.4312    |
| 61        | 0.975     | 18.59     | 0.4772    | 7.264     | 435.6     | 0.7924    |
| 62        | 0.97      | 9.781     | 0.1092    | 3.248     | 117.7     | 0.1038    |
| 63        | 0.9736    | 45.28     | 0.4901    | 6.569     | 250.9     | 0.6162    |
| 64        | 0.9737    | 7.079     | 0.3497    | 6.076     | 345.1     | 0.7173    |
| 65        | 0.9727    | 27.87     | 0.1533    | 5.304     | 144.7     | 0.7087    |
| 66        | 0.9745    | 20.8      | 0.3552    | 5.013     | 178.8     | 0.3       |
| 67        | 0.9742    | 25.77     | 0.461     | 8.052     | 170.1     | 0.0337    |
| 68        | 0.9752    | 22.67     | 0.1456    | 8.403     | 230.3     | 0.1676    |
| 69        | 0.9742    | 44.53     | 0.4049    | 5.395     | 179.5     | 0.5579    |
| 70        | 0.9755    | 38.79     | 0.3514    | 9.15

| 117       | 0.9756    | 4.435     | 0.1745    | 7.126     | 372.7     | 0.1458    |
| 118       | 0.9531    | 26.13     | 0.00447   | 9.288     | 424.7     | 0.1655    |
| 119       | 0.9755    | 4.051     | 0.1852    | 6.66      | 372.8     | 0.7373    |
| 120       | 0.9752    | 46.12     | 0.2889    | 8.484     | 363.1     | 0.1357    |
| 121       | 0.9746    | 46.07     | 0.3548    | 7.638     | 494.3     | 0.5338    |
| 122       | 0.9772    | 1.929     | 0.2464    | 9.697     | 291.9     | 0.3514    |
| 123       | 0.9763    | 4.157     | 0.21      | 7.541     | 373.9     | 0.7419    |
| 124       | 0.9751    | 2.98      | 0.2879    | 7.001     | 372.8     | 0.5096    |
| 125       | 0.9737    | 2.865     | 0.03576   | 8.428     | 292.9     | 0.8287    |
| 126       | 0.9755    | 1.004     | 0.3062    | 9.571     | 291.9     | 0.2695    |
| 127       | 0.9764    | 1.009     | 0.1302    | 9.188     | 358.5     | 0.8306    |
| 128       | 0.975     | 1.318     | 0.4427    | 9.43

| 175       | 0.9736    | 18.25     | 0.03035   | 9.985     | 279.9     | 0.4545    |
| 176       | 0.971     | 22.47     | 0.2191    | 3.077     | 109.1     | 0.5347    |
| 177       | 0.9748    | 17.78     | 0.4574    | 7.654     | 281.0     | 0.6082    |
| 178       | 0.9741    | 13.26     | 0.4208    | 4.439     | 336.9     | 0.4135    |
| 179       | 0.9743    | 2.367     | 0.4799    | 9.302     | 358.2     | 0.4284    |
| 180       | 0.9734    | 38.97     | 0.07933   | 9.875     | 214.0     | 0.7928    |
| 181       | 0.9759    | 1.962     | 0.1515    | 8.232     | 289.8     | 0.2042    |
| 182       | 0.9762    | 18.57     | 0.2731    | 9.166     | 278.5     | 0.3432    |
| 183       | 0.9755    | 46.71     | 0.1087    | 8.05      | 493.8     | 0.7028    |
| 184       | 0.9746    | 38.71     | 0.08636   | 8.637     | 314.7     | 0.07274   |
| 185       | 0.9723    | 16.99     | 0.02579   | 8.586     | 280.8     | 0.4458    |
| 186       | 0.9755    | 45.7      | 0.3674    | 7.87

| 233       | 0.9739    | 21.57     | 0.485     | 6.858     | 263.1     | 0.09545   |
| 234       | 0.9766    | 1.392     | 0.1942    | 8.668     | 289.5     | 0.08316   |
| 235       | 0.9736    | 14.7      | 0.1374    | 5.246     | 133.0     | 0.1908    |
| 236       | 0.9736    | 45.17     | 0.3711    | 4.168     | 481.0     | 0.3645    |
| 237       | 0.9743    | 48.25     | 0.3601    | 8.591     | 151.1     | 0.4667    |
| 238       | 0.9766    | 1.923     | 0.2381    | 8.752     | 289.1     | 0.7409    |
| 239       | 0.9749    | 9.467     | 0.1918    | 5.751     | 393.6     | 0.4987    |
| 240       | 0.9744    | 43.94     | 0.4399    | 4.718     | 311.4     | 0.3069    |
| 241       | 0.9748    | 2.754     | 0.03164   | 9.553     | 292.7     | 0.06394   |
| 242       | 0.9751    | 44.47     | 0.2171    | 7.795     | 493.9     | 0.06229   |
| 243       | 0.9751    | 46.53     | 0.4943    | 8.265     | 494.3     | 0.8398    |
| 244       | 0.9763    | 1.723     | 0.3231    | 9.72

In [14]:
# XGB Maximize
BO_xgb_clf.maximize(init_points=50, n_iter=150, acq = 'poi')

|   iter    |  target   | colsam... |   gamma   | learni... | max_depth | min_ch... | n_esti... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------
| 1         | 0.9708    | 0.7884    | 5.311     | 0.09657   | 3.475     | 12.02     | 345.4     | 6.379     | 5.76      | 0.7117    |
| 2         | 0.9532    | 0.8073    | 9.457     | 0.03096   | 9.048     | 13.28     | 73.04     | 6.528     | 5.522     | 0.8793    |
| 3         | 0.9748    | 0.8451    | 2.837     | 0.1496    | 6.931     | 6.545     | 404.9     | 4.191     | 1.448     | 0.7453    |
| 4         | 0.9726    | 0.7166    | 7.183     | 0.1469    | 4.391     | 12.64     | 305.6     | 0.8326    | 5.455     | 0.7477    |
| 5         | 0.9725    | 0.903     | 1.194     | 0.2231    | 9.216     | 12.16     | 80.59     | 9.608     | 6.595     | 0.9156    |
| 6         | 0.9731    | 0.9231    | 8.878     | 0.06784   | 

| 37        | 0.9722    | 0.9391    | 5.488     | 0.1533    | 5.304     | 3.946     | 404.4     | 4.048     | 7.102     | 0.7863    |
| 38        | 0.9747    | 0.7858    | 3.34      | 0.2532    | 9.453     | 11.1      | 170.1     | 0.384     | 4.428     | 0.7869    |
| 39        | 0.974     | 0.9316    | 4.012     | 0.09393   | 9.218     | 12.33     | 204.0     | 2.884     | 6.203     | 0.9314    |
| 40        | 0.9739    | 0.9106    | 8.793     | 0.2943    | 4.977     | 9.462     | 340.0     | 0.7467    | 0.9894    | 0.8237    |
| 41        | 0.9746    | 0.8013    | 3.453     | 0.2471    | 5.214     | 3.763     | 247.3     | 1.906     | 6.866     | 0.8188    |
| 42        | 0.9731    | 0.8097    | 5.957     | 0.07125   | 7.544     | 2.638     | 415.9     | 4.743     | 0.7504    | 0.8307    |
| 43        | 0.9714    | 0.8487    | 4.266     | 0.05128   | 9.56      | 7.341     | 461.7     | 9.833     | 4.185     | 0.9853    |
| 44        | 0.9723    | 0.8184    | 4.033     | 0.04841   | 

| 74        | 0.9749    | 0.9658    | 1.716     | 0.2157    | 7.208     | 6.075     | 405.3     | 2.532     | 2.498     | 0.7547    |
| 75        | 0.9383    | 0.7646    | 2.31      | 0.001216  | 6.293     | 3.762     | 246.8     | 2.707     | 5.811     | 0.9249    |
| 76        | 0.9733    | 0.7551    | 2.109     | 0.3882    | 6.785     | 14.24     | 168.1     | 5.187     | 8.813     | 0.7337    |
| 77        | 0.9732    | 0.864     | 6.886     | 0.06715   | 6.487     | 8.187     | 338.6     | 1.507     | 6.243     | 0.8767    |
| 78        | 0.9709    | 0.9026    | 8.195     | 0.3745    | 6.534     | 12.41     | 435.5     | 3.711     | 1.161     | 0.7187    |
| 79        | 0.9727    | 0.8294    | 2.904     | 0.4762    | 6.766     | 6.701     | 404.2     | 4.607     | 1.143     | 0.7345    |
| 80        | 0.9743    | 0.8797    | 0.1582    | 0.3919    | 9.658     | 10.54     | 426.6     | 1.019     | 8.074     | 0.8335    |
| 81        | 0.9742    | 0.806     | 3.506     | 0.3183    | 

| 111       | 0.9753    | 0.9079    | 1.679     | 0.1713    | 7.342     | 5.778     | 404.7     | 3.917     | 0.4521    | 0.8816    |
| 112       | 0.9758    | 0.7807    | 1.099     | 0.206     | 7.783     | 5.639     | 404.8     | 3.448     | 2.583     | 0.9521    |
| 113       | 0.9739    | 0.8499    | 6.035     | 0.3013    | 9.178     | 3.151     | 415.7     | 3.853     | 0.3479    | 0.8482    |
| 114       | 0.9733    | 0.9082    | 7.132     | 0.2147    | 6.14      | 13.16     | 436.3     | 2.305     | 1.004     | 0.9025    |
| 115       | 0.9752    | 0.9713    | 0.9514    | 0.08108   | 8.246     | 4.491     | 403.7     | 4.761     | 1.767     | 0.9489    |
| 116       | 0.9749    | 0.8314    | 0.5438    | 0.141     | 6.781     | 6.527     | 406.2     | 4.344     | 3.388     | 0.771     |
| 117       | 0.9742    | 0.9411    | 2.088     | 0.3671    | 8.752     | 7.546     | 298.5     | 6.723     | 9.699     | 0.902     |
| 118       | 0.9737    | 0.8439    | 3.225     | 0.4266    | 

E:\Anaconda\lib\site-packages\joblib\externals\loky\process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


| 120       | 0.9749    | 0.9627    | 0.1561    | 0.1064    | 7.044     | 6.021     | 407.1     | 3.32      | 3.376     | 0.8046    |
| 121       | 0.9751    | 0.7445    | 1.987     | 0.2922    | 7.19      | 5.127     | 404.8     | 2.339     | 2.404     | 0.9908    |
| 122       | 0.9737    | 0.7211    | 6.834     | 0.332     | 5.87      | 12.09     | 436.0     | 2.593     | 0.7125    | 0.8883    |
| 123       | 0.9753    | 0.7979    | 3.57      | 0.1006    | 7.864     | 6.491     | 403.6     | 2.944     | 3.009     | 0.9265    |
| 124       | 0.9752    | 0.876     | 2.285     | 0.1163    | 7.165     | 6.86      | 406.9     | 4.04      | 4.754     | 0.8956    |
| 125       | 0.9736    | 0.9375    | 2.07      | 0.02305   | 6.6       | 7.151     | 407.6     | 4.469     | 4.219     | 0.9108    |
| 126       | 0.9747    | 0.7474    | 3.587     | 0.2357    | 8.849     | 9.857     | 169.4     | 0.6686    | 4.447     | 0.7724    |
| 127       | 0.9727    | 0.9512    | 1.578     | 0.4849    | 

KeyboardInterrupt: 

In [ ]:
# LGBM Maximize
BO_lgbm_clf.maximize(init_points=50, n_iter=150, acq = 'poi')

In [ ]:
# ExtraTrees Maximize
BO_et_clf.maximize(init_points=50, n_iter=150, acq = 'poi')

In [ ]:
# Logistic Maximize
BO_lr_clf.maximize(init_points=50, n_iter=150, acq = 'poi')

### Change Max parameters from float to int

In [ ]:
# Catboost Max Parameters
cb_clf_max_params = BO_cb_clf.max['params']
cb_clf_max_params['n_estimators'] = int(round(cb_clf_max_params['n_estimators']))
cb_clf_max_params['max_depth'] = int(round(cb_clf_max_params['max_depth']))

# XGB Max Parameters
xgb_clf_max_params = BO_xgb_clf.max['params']
xgb_clf_max_params['n_estimators'] = int(round(xgb_clf_max_params['n_estimators']))
xgb_clf_max_params['max_depth'] = int(round(xgb_clf_max_params['max_depth']))

In [ ]:
# LGBM Max Parameters
lgbm_clf_max_params = BO_lgbm_clf.max['params']
lgbm_clf_max_params['n_estimators'] = int(round(lgbm_clf_max_params['n_estimators']))
lgbm_clf_max_params['max_depth'] = int(round(lgbm_clf_max_params['max_depth']))
lgbm_clf_max_params['min_child_samples'] = int(round(lgbm_clf_max_params['min_child_samples']))

# ExtraTrees Max Parameters
et_clf_max_params = BO_et_clf.max['params']
et_clf_max_params['n_estimators'] = int(round(et_clf_max_params['n_estimators']))
et_clf_max_params['max_depth'] = int(round(et_clf_max_params['max_depth']))
et_clf_max_params['min_samples_split'] = int(round(et_clf_max_params['min_samples_split']))
et_clf_max_params['min_samples_leaf'] = int(round(et_clf_max_params['min_samples_leaf']))

# Logistic Max Parameters
lr_clf_max_params = BO_lr_clf.max['params']
lr_clf_max_params['max_iter'] = int(round(lr_clf_max_params['max_iter']))

### Retraing the Model with best parameters

In [ ]:
tune_cb_clf = CatBoostClassifier(**cb_clf_max_params)
tune_cb_clf.fit(tomek_x_train,tomek_y_train)

tune_xgb_clf = XGBClassifier(**xgb_clf_max_params)
tune_xgb_clf.fit(tomek_x_train,tomek_y_train)

In [ ]:
tune_lgbm_clf = LGBMClassifier(**lgbm_clf_max_params)
tune_lgbm_clf.fit(tomek_x_train,tomek_y_train)

tune_et_clf = ExtraTreesClassifier(**et_clf_max_params)
tune_et_clf.fit(tomek_x_train,tomek_y_train)

tune_lr_clf = LogisticRegression(**lr_clf_max_params)
tune_lr_clf.fit(tomek_x_train,tomek_y_train)

### Validation Check

In [ ]:
# Predict x_val by Tuned Model
tune_cb_clf_pred = tune_cb_clf.predict(x_val)
tune_xgb_clf_pred = tune_xgb_clf.predict(x_val)

In [ ]:
tune_lgbm_clf_pred = tune_lgbm_clf.predict(x_val)
tune_et_clf_pred = tune_et_clf.predict(x_val)
tune_lr_clf_pred = tune_lr_clf.predict(x_val)

In [ ]:
# Validation Score
tune_cb_clf_val_score = f1_score(y_val, tune_cb_clf_pred, average = 'macro')
tune_xgb_clf_val_score = f1_score(y_val, tune_xgb_clf_pred, average = 'macro')
#tune_lgbm_clf_val_score = f1_score(y_val, tune_lgbm_clf_pred, average = 'macro')
#tune_et_clf_val_score = f1_score(y_val, tune_et_clf_pred, average = 'macro')
#tune_lr_clf_val_score = f1_score(y_val, tune_lr_clf_pred, average = 'macro')
print(f'Catboost F1 score:{tune_cb_clf_val_score}\nXGB F1 score:{tune_xgb_clf_val_score}')
#print(f'LGBM F1 score:{tune_lgbm_clf_val_score}\nExtraTrees F1 score:{tune_et_clf_val_score}\nLogistic F1 score:{tune_lr_clf_val_score}')

### Generate predict_proba

In [ ]:
tune_cb_clf_pred_proba = np.arrary(i[1] for i in tune_cb_clf.predict_proba(tomek_x_train))
tune_xgb_clf_pred_proba = np.arrary(i[1] for i in tune_xgb_clf.predict_proba(tomek_x_train))
#tune_lgbm_clf_pred_proba = np.arrary(i[1] for i in tune_lgbm_clf.predict_proba(tomek_x_train))
#tune_et_clf_pred_proba = np.arrary(i[1] for i in tune_et_clf.predict_proba(tomek_x_train))
#tune_lr_clf_pred_proba = np.arrary(i[1] for i in tune_lr_clf.predict_proba(tomek_x_train))

## Saving Task1 Model

In [ ]:
joblib.dump(tune_cb_clf, 'model_save/tuning/task1_cb_clf.pkl')
joblib.dump(tune_xgb_clf, 'model_save/tuning/task1_xgb_clf.pkl')
#joblib.dump(tune_lgbm_clf, 'model_save/tuning/task1_lgbm_clf.pkl')
#joblib.dump(tune_et_clf, 'model_save/tuning/task1_et_clf.pkl')
#joblib.dump(tune_lr_clf, 'model_save/tuning/task1_lr_clf.pkl')

# Task2 : Predict Test Label by Regressor

### Generate Functoin for Bayesian Optimizer (Regressor)

In [ ]:
def cb_reg_opt(n_estimators, learning_rate, max_depth, random_strength, l2_leaf_reg):
    
    params = { 'n_estimators' : int(round(n_estimators)),
               'learning_rate' : learning_rate,
               'max_depth' : int(round(max_depth)),
               'random_strength' : random_strength,
               'l2_leaf_reg' : l2_leaf_reg }
    
    cb = CatBoostRegressor(**params)
    skf = KFold(n_splits = 5, shuffle = True, random_state = 17)
    score = cross_val_score(cb, tomek_s_train, ***, scoring = 'neg_mean_squared_error', cv = skf, n_jobs = -1)
    
    return np.mean(score)

def xgb_reg_opt(n_estimators, learning_rate, max_depth, subsample, colsample_bytree, 
                reg_alpha, reg_lambda, gamma, min_child_weight):
    
    params = { 'n_estimators' : int(round(n_estimators)),
               'learning_rate' : learning_rate,
               'max_depth' : int(round(max_depth)),
               'subsample' : subsample,
               'colsample_bytree' : colsample_bytree,
               'reg_alpha' : reg_alpha,
               'reg_lambda' : reg_lambda,
               'gamma' : gamma,
               'min_child_weight' : min_child_weight, 'n_jobs' : -1 }
    
    xgb = XGBRegressor(**params)
    skf = KFold(n_splits = 5, shuffle = True, random_state = 17)
    score = cross_val_score(xgb, tomek_s_train, ***, scoring = 'neg_mean_squared_error', cv = skf, n_jobs = -1)
    
    return np.mean(score)

def lgbm_reg_opt(n_estimators, learning_rate, max_depth, min_child_samples, subsample, colsample_bytree, 
                reg_alpha, reg_lambda, min_child_weight, min_split_gain):
    
    params = { 'n_estimators' : int(round(n_estimators)),
               'learning_rate' : learning_rate,
               'max_depth' : int(round(max_depth)),
               'min_child_samples' : int(round(min_child_samples)),
               'subsample' : subsample,
               'colsample_bytree' : colsample_bytree,
               'reg_alpha' : reg_alpha,
               'reg_lambda' : reg_lambda,
               'min_child_weight' : min_child_weight,
               'min_split_gain' : min_split_gain, 'n_jobs' : -1 }
    
    lgbm = LGBMRegressor(**params)
    skf = KFold(n_splits = 5, shuffle = True, random_state = 17)
    score = cross_val_score(lgbm, tomek_s_train, ***, scoring = 'neg_mean_squared_error', cv = skf, n_jobs = -1)
    
    return np.mean(score)

def et_reg_opt(n_estimators, max_depth, min_samples_split, min_samples_leaf):
    
    params = { 'n_estimators' : int(round(n_estimators)),
               'max_depth' : int(round(max_depth)),
               'min_samples_split' : int(round(min_samples_split)),
               'min_samples_leaf' : int(round(min_samples_leaf)), 'n_jobs' : -1 }
    
    et = ExtraTreesRegressor(**params)
    skf = KFold(n_splits = 5, shuffle = True, random_state = 17)
    score = cross_val_score(et, tomek_s_train, ***, scoring = 'neg_mean_squared_error', cv = skf, n_jobs = -1)
    
    return np.mean(score)

def lr_reg_opt(n_estimators, C):
    
    params = { 'max_iter' : int(round(max_iter)), 'C' : C, 'n_jobs' : -1 }
    
    lr = LogisticRegression(**params)
    skf = KFold(n_splits = 5, shuffle = True, random_state = 17)
    score = cross_val_score(lr, tomek_s_train, ***, scoring = 'neg_mean_squared_error', cv = skf, n_jobs = -1)
    
    return np.mean(score)

### Generate BayesianOptimization

In [ ]:
BO_cb_reg = BayesianOptimization(f = cb_reg_opt, pbounds = cb_pbounds, random_state=17)
BO_xgb_reg = BayesianOptimization(f = xgb_reg_opt, pbounds = xgb_pbounds, random_state=17)
BO_lgbm_reg = BayesianOptimization(f = lgbm_reg_opt, pbounds = lgbm_pbounds, random_state=17)
BO_et_reg = BayesianOptimization(f = et_reg_opt, pbounds = et_pbounds, random_state=17)
BO_lr_reg = BayesianOptimization(f = lr_reg_opt, pbounds = lr_pbounds, random_state=17)

### Optimizer Maximize

In [ ]:
BO_cb_reg.maximize(init_points=50, n_iter=150, acq = 'ei', xi = 0.01)
BO_xgb_reg.maximize(init_points=50, n_iter=150, acq = 'ei', xi = 0.01)
BO_lgbm_reg.maximize(init_points=50, n_iter=150, acq = 'ei', xi = 0.01)
BO_et_reg.maximize(init_points=50, n_iter=150, acq = 'ei', xi = 0.01)
BO_lr_reg.maximize(init_points=50, n_iter=150, acq = 'ei', xi = 0.01)

### Change Max parameters from float to int

In [ ]:
# Catboost Max Parameters
cb_reg_max_params = BO_cb_reg.max['params']
cb_reg_max_params['n_estimators'] = int(round(cb_reg_max_params['n_estimators']))
cb_reg_max_params['max_depth'] = int(round(cb_reg_max_params['max_depth']))

# XGB Max Parameters
xgb_reg_max_params = BO_xgb_reg.max['params']
xgb_reg_max_params['n_estimators'] = int(round(xgb_reg_max_params['n_estimators']))
xgb_reg_max_params['max_depth'] = int(round(xgb_reg_max_params['max_depth']))

# LGBM Max Parameters
lgbm_reg_max_params = BO_lgbm_reg.max['params']
lgbm_reg_max_params['n_estimators'] = int(round(lgbm_reg_max_params['n_estimators']))
lgbm_reg_max_params['max_depth'] = int(round(lgbm_reg_max_params['max_depth']))
lgbm_reg_max_params['min_child_samples'] = int(round(lgbm_reg_max_params['min_child_samples']))

# ExtraTrees Max Parameters
et_reg_max_params = BO_et_reg.max['params']
et_reg_max_params['n_estimators'] = int(round(et_reg_max_params['n_estimators']))
et_reg_max_params['max_depth'] = int(round(et_reg_max_params['max_depth']))
et_reg_max_params['min_samples_split'] = int(round(et_reg_max_params['min_samples_split']))
et_reg_max_params['min_samples_leaf'] = int(round(et_reg_max_params['min_samples_leaf']))

# Logistic Max Parameters
lr_reg_max_params = BO_lr_reg.max['params']
lr_reg_max_params['max_iter'] = int(round(lr_reg_max_params['max_iter']))

### Retraing the Model with best parameters

In [ ]:
tune_cb_reg = CatBoostRegressor(**cb_reg_max_params)
tune_cb_reg.fit(tomek_s_train,tomek_y_train)

tune_xgb_reg = XGBRegressor(**xgb_reg_max_params)
tune_xgb_reg.fit(tomek_s_train,tomek_y_train)

tune_lgbm_reg = LGBMRegressor(**lgbm_reg_max_params)
tune_lgbm_reg.fit(tomek_s_train,tomek_y_train)

tune_et_reg = ExtraTreesRegressor(**et_reg_max_params)
tune_et_reg.fit(tomek_s_train,tomek_y_train)

tune_lr_reg = LogisticRegressionr(**lr_reg_max_params)
tune_lr_reg.fit(tomek_s_train,tomek_y_train)

### Validation Check

In [ ]:
# Predict x_s_val by Tuned Model
tune_cb_reg_val_pred = tune_cb_reg.predict(x_s_val)
tune_xgb_reg_val_pred = tune_xgb_reg.predict(x_s_val)
tune_lgbm_reg_val_pred = tune_lgbm_reg.predict(x_s_val)
tune_et_reg_val_pred = tune_et_reg.predict(x_s_val)
tune_lr_reg_val_pred = tune_lr_reg.predict(x_s_val)

In [ ]:
# Visualize Validation predict
cb_val_pred = pd.DataFrame({'predict':tune_cb_reg_val_pred})
xgb_val_pred = pd.DataFrame({'predict':tune_xgb_reg_val_pred})
lgbm_val_pred = pd.DataFrame({'predict':tune_lgbm_reg_val_pred})
et_val_pred = pd.DataFrame({'predict':tune_et_reg_val_pred})
lr_val_pred = pd.DataFrame({'predict':tune_lr_reg_val_pred})

plt.figure(figsize=(20,10))

plt.subplot(3,2,1)
sns.kdeplot(cb_val_pred.predict)
plt.title('Caboost Predict KDE (val)')
plt.show()

plt.subplot(3,2,2)
sns.kdeplot(xgb_val_pred.predict)
plt.title('XGB Predict KDE (val)')
plt.show()

plt.subplot(3,2,3)
sns.kdeplot(lgbm_val_pred.predict)
plt.title('LGBM Predict KDE (val)')
plt.show()

plt.subplot(3,2,4)
sns.kdeplot(et_val_pred.predict)
plt.title('ExtraTrees Predict KDE (val)')
plt.show()

plt.subplot(3,2,5)
sns.kdeplot(lr_val_pred.predict)
plt.title('Logistic Predict KDE (val)')
plt.show()

In [ ]:
# Check 8.5% raw at Validation predict
print(cb_val_pred.predict.sort_values(ascending = False).iloc[510:521])
print(xgb_val_pred.predict.sort_values(ascending = False).iloc[510:521])
print(lgbm_val_pred.predict.sort_values(ascending = False).iloc[510:521])
print(et_val_pred.predict.sort_values(ascending = False).iloc[510:521])
print(lr_val_pred.predict.sort_values(ascending = False).iloc[510:521])

In [ ]:
# predict transform by threshold
cb_trans_val_pred = np.array([0 if i < * else 1 for i in tune_cb_reg_val_pred])
xgb_trans_val_pred = np.array([0 if i < * else 1 for i in tune_xgb_reg_val_pred])
lgbm_trans_val_pred = np.array([0 if i < * else 1 for i in tune_lgbm_reg_val_pred])
et_trans_val_pred = np.array([0 if i < * else 1 for i in tune_et_reg_val_pred])
lr_trans_val_pred = np.array([0 if i < * else 1 for i in tune_lr_reg_val_pred])

In [ ]:
# Validation F1-macro Score & ConfusionMatrix
tune_cb_reg_val_score = f1_score(y_val, cb_trans_val_pred, average = 'macro')
tune_xgb_reg_val_score = f1_score(y_val, xgb_trans_val_pred, average = 'macro')
tune_lgbm_reg_val_score = f1_score(y_val, tune_lgbm_reg_pred, average = 'macro')
tune_et_reg_val_score = f1_score(y_val, et_trans_val_pred, average = 'macro')
tune_lr_reg_val_score = f1_score(y_val, lr_trans_val_pred, average = 'macro')

print(f'Catboost F1 score : {tune_cb_reg_val_score}\n', classification_report(y_val, cb_trans_val_pred))
print(f'XGB F1 score : {tune_xgb_reg_val_score}\n', classification_report(y_val, xgb_trans_val_pred))
print(f'LGBM F1 score : {tune_lgbm_reg_val_score}\n', classification_report(y_val, lgbm_trans_val_pred))
print(f'ExtraTrees F1 score : {tune_et_reg_val_score}\n', classification_report(y_val, et_trans_val_pred))
print(f'Logistic F1 score : {tune_lr_reg_val_score}\n', classification_report(y_val, lr_trans_val_pred))

### Generate Test Predict

In [ ]:
# Predict target by test_x
tune_cb_reg_pred = tune_cb_reg.predict(test_x)
tune_xgb_reg_pred = tune_xgb_reg.predict(test_x)
tune_lgbm_reg_pred = tune_lgbm_reg.predict(test_x)
tune_et_reg_pred = tune_et_reg.predict(test_x)
tune_lr_reg_pred = tune_lr_reg.predict(test_x)

In [ ]:
# Visualize target predict
cb_pred = pd.DataFrame({'predict':tune_cb_reg_pred})
xgb_pred = pd.DataFrame({'predict':tune_xgb_reg_pred})
lgbm_pred = pd.DataFrame({'predict':tune_lgbm_reg_pred})
et_pred = pd.DataFrame({'predict':tune_et_reg_pred})
lr_pred = pd.DataFrame({'predict':tune_lr_reg_pred})

plt.figure(figsize=(20,10))

plt.subplot(3,2,1)
sns.kdeplot(cb_pred.predict)
plt.title('Caboost Predict KDE (predict)')
plt.show()

plt.subplot(3,2,2)
sns.kdeplot(xgb_pred.predict)
plt.title('XGB Predict KDE (predict)')
plt.show()

plt.subplot(3,2,3)
sns.kdeplot(lgbm_pred.predict)
plt.title('LGBM Predict KDE (predict)')
plt.show()

plt.subplot(3,2,4)
sns.kdeplot(et_pred.predict)
plt.title('ExtraTrees Predict KDE (predict)')
plt.show()

plt.subplot(3,2,5)
sns.kdeplot(lr_pred.predict)
plt.title('Logistic Predict KDE (predict)')
plt.show()

In [ ]:
# target predict 8.5% raw print by Model
print(cb_pred.predict.sort_values(ascending = False).iloc[510:521])
print(xgb_pred.predict.sort_values(ascending = False).iloc[510:521])
print(lgbm_pred.predict.sort_values(ascending = False).iloc[510:521])
print(et_pred.predict.sort_values(ascending = False).iloc[510:521])
print(lr_pred.predict.sort_values(ascending = False).iloc[510:521])

In [ ]:
# Transform predict by Threshold
cb_trans_reg_pred = np.array([0 if i < * else 1 for i in tune_cb_reg_pred])
xgb_trans_reg_pred = np.array([0 if i < * else 1 for i in tune_xgb_reg_pred])
lgbm_trans_reg_pred = np.array([0 if i < * else 1 for i in tune_lgbm_reg_pred])
et_trans_reg_pred = np.array([0 if i < * else 1 for i in tune_et_reg_pred])
lr_trans_reg_pred = np.array([0 if i < * else 1 for i in tune_lr_reg_pred])

## Saving Task1 Model

In [ ]:
# Saving Model
joblib.dump(tune_cb_reg, 'model_save/tuning/task1_xxx_clf_task2_cb_reg.pkl')
joblib.dump(tune_xgb_reg, 'model_save/tuning/task1_xxx_clf_task2_xgb_reg.pkl')
joblib.dump(tune_lgbm_reg, 'model_save/tuning/task1_xxx_clf_task2_lgbm_reg.pkl')
joblib.dump(tune_rt_reg, 'model_save/tuning/task1_xxx_clf_task2_rt_reg.pkl')
joblib.dump(tune_lr_reg, 'model_save/tuning/task1_xxx_clf_task2_lr_reg.pkl')

## to_csv Submission

In [ ]:
pd.DataFrame({'ID': test_id, 'Y_LABEL': cb_trans_reg_pred}).to_csv('predict/modeling/xx_clf_cb_reg_.csv', index=False)
pd.DataFrame({'ID': test_id, 'Y_LABEL': xgb_trans_reg_pred}).to_csv('predict/modeling/xx_clf_xgb_reg_.csv', index=False)
pd.DataFrame({'ID': test_id, 'Y_LABEL': lgbm_trans_reg_pred}).to_csv('predict/modeling/xx_clf_lgbm_reg_.csv', index=False)
pd.DataFrame({'ID': test_id, 'Y_LABEL': et_trans_reg_pred}).to_csv('predict/modeling/xx_clf_et_reg_.csv', index=False)
pd.DataFrame({'ID': test_id, 'Y_LABEL': lr_trans_reg_pred}).to_csv('predict/modeling/xx_clf_lr_reg_.csv', index=False)